# Stroj na předpověď

Principy, které si nyní ukážeme jsou aplikované při konstrukci všech supervised machile learning modelů. Náš velmi zjednodušený supervised machine learning model bude mít jeden vstup a jeden výstup. Jeho úkolem je najít hodnotu W, která odpovídá konstantě 62.13712, která se používá při převodu kilometrů na míle. Funkce konverze je lineární y = x * W.

Zažité je označovat vstup modelu jako X, požadovaný výstup modelu Y a předpověď modelu ŷ [y-hat]. Pro určení správné hodnoty W využijeme funkce Tensorflow a to funkce optimalizační. 

Dá se říci, že čím je funkce, na kterou chceme model trénovat, komplexnější, tím více vzorků dané funkce potřebujeme použít při tréningu. S rostoucí komplexitou funkce roste i komplexita modelu - např. více neuronů, více vrstev, kombinace různých aktivačních funkcí, atd... V naší ukázce převodu kilometrů na míle nám by nám stačilo minimální množství vzorků, resp. pro správné určení hodnoty W by nám ¨stačil jediný vzorek, kde X != 0. Abychom se v ukázce více přiblížili reálnému procesu učení, vzorků použijeme více.

![alt text](pictures/machine_learning_small.jpg  "Machine Learning")

Toto jsou naše vzorky funkce v X a Y.

In [8]:
import tensorflow as tf

nr_inputs = 1

X =   [ [8.],       [45.],      [323.],     [0.0], [100] ]
Y =   [ [4.970970], [27.96170], [200.7029], [0.0], [62.13712] ]

<b>Vzorky funkce, které máme k dispozici, rozdělíme.</b> Část z nich použijeme na učení našeho modelu, druhou si necháme k tomu, abychom si ověřili, zda model je schopen provádět předpověď i nad daty, která nebyla použita při tréningu.

In [3]:
train_x =   X[:3]
train_y =   Y[:3]
print('train_x: ', train_x)
print('train_y: ', train_y)

test_x  =   X[3:]
test_y  =   Y[3:]
print('test_x: ', test_x)
print('test_y: ', test_y)

train_x:  [[8.0], [45.0], [323.0]]
train_y:  [[4.97097], [27.9617], [200.7029]]
test_x:  [[0.0], [100]]
test_y:  [[0.0], [62.13712]]


Nyní připravíme Tensory, které budeme pro výpočet v grafu potřebovat. Ve scope 'input' definujeme placeholder pro vložení vstupních dat to grafu, ve scope 'target' pak placeholder, kde předáváme modelu 'label', tedy správný výsledek k příslušnému vstupu. Definice scope není povinná, má zde jen pomoci lepší přehlednosti grafu zobrazeného na Tensorboardu.

In [4]:
with tf.name_scope('input'):
   x = tf.placeholder(tf.float32, [None, nr_inputs], name='input')

with tf.name_scope('target'):
   y = tf.placeholder(tf.float32, [None, 1], name='target')

Zde je vnitřek našeho 'prediction machine' modelu. Za sofistikovaným algoritmem strojového učení se v našem zjednodušeném příkladu skrývá pouze lineární funkce x * W, ale jak uvidíte později, každý neuron použitý v neuronových sítích, má v sobě rovněž obyčejnou lineární funkci. Tato funkce mu umožňuje provádět klasifikaci. 

Tensor Variable je pro Tensorflow tzv. Trainable Tensor. Trainable Tensory mohou, a jsou, měněny funkcemi Tensorflow - optimalizačnímy algoritmy. Optimalizační algoritmus má za úkol provést takové změny hodnot Tensoru/Tensorů typu Variable, aby funkce, ve které jsou Tensory typu Variable použity, konvergovala k co nejrychleji k co nejmenší hodnotě chyby. Algoritmus učení našeho modelu je zachycen na následujícím obrázku:

![alt text](pictures/ML_pattern.png "Suervised ML Pattern")

V prvním kroku, na obrázku zelenou barvou, <font color='green'>do Grafu vložíme pomocí Tensoru typu placeholder naše vzorky dat X a Y</font>. Nad těmito daty spustíme výpočet, což znamená, že <font color='blue'>voláme funkci train</font> - je definována v modrém obdélníčku. Funkce <font color='red'>train</font> spouští 'cost_function', na obrázku červená, která jako parametr používá funckci prediction, žlutě. Na tomto případě je vidět vzájemná provázanost jednotlivých částí grafu. 

In [5]:
with tf.name_scope('p_machine'):

    with tf.name_scope('Wtrainable'):
        # Weight - Variable is by default trainable
        W = tf.Variable(tf.random_uniform([nr_inputs, 1], -1, 1), name="rand_W")
        tf.trainable_variables()
    
    with tf.name_scope('prediction_fn'):
        pred = tf.matmul(x, W, name='Wx')

Jak se Tensorflow přilíží ke správnému výsledku? Tensorflow provede úpravu hodnoty tensoru W a srovnáním se správným výsledkem (pred - y) spočítá, zda se hodnota chyby zvětšila, nebo zmenšila. Potom se vydá v optimalizaci hodnoty tensoru W takovým směrem, aby hodnota chyby po další iteraci klesla. Algoritmus provádějící optimalizaci parametrů modelu se nazývá <b>Gradient Descent</b>. Parametr 'learning_rate' zásadně ovlivňuje proces učení, optimalizačnímu algoritmu říká, v jak velkých krocích má postupovat při úpravě našeho Variable Tensoru. 

In [6]:
with tf.name_scope("training") as scope:
    
    with tf.name_scope("cost_fn") as scope:
        cost_function = tf.reduce_sum(pred - y) ** 2
        
    #train = tf.train.AdamOptimizer().minimize(cost_function)
    #vyzkoušej  
    #train = tf.train.GradientDescentOptimizer(0.01).minimize(cost_function)
    #výpočet by měl selhat, hodnota NaN, příliš velký slope 
    train = tf.train.GradientDescentOptimizer(0.000001).minimize(cost_function)

Pokud je parametr 'learning_rate' příliš velký, optimalizační algoritmus nemusí konvergovat ke správnému výsledku - velká hodnota parametru 'learning_rate' by 'přeskočila' tzv. globální minimum, kde je hodnota chyby (cost) predikce našeho modelu nejmenší. Ukázka chování optimalizačního algoritmu v případě, kdy je parametr 'learning_rate' příliš velký, je vidět na obrázku B.

![alt text](pictures/gradient_descent_slope.png "Gradient Descent slope")

Možná trochu matoucí názvy <b>'cost'</b>, případně <b>'loss'</b> jsou v ML zažité termíny pro <b>chybu</b>.

Tady už vše necháme na Tensorflow. V momentě, kdy bude náš model generovat dostatečně malou chybu, proces učení ukončíme. 

In [7]:
# loop through the Graph until the error is low enough
with tf.Session() as sess:
    tb_writer = tf.summary.FileWriter("./tensorboard_example", sess.graph)
    tb_scalar_cost = tf.summary.scalar("cost", cost_function)
 
    sess.run(tf.global_variables_initializer())

    cost_function_result = 1000.0
    training_epoch = -1
    
    while cost_function_result > 0.01:
        
        training_epoch += 1
        
        _, tb_scalar_cost_report, cost_function_result = \
            sess.run([train, tb_scalar_cost, cost_function], feed_dict={x: train_x, y: train_y})
        tb_writer.add_summary(tb_scalar_cost_report, training_epoch)
        
        if training_epoch % 1000 == 0:
            print('epoch: ', training_epoch, ' cost: ', cost_function_result)
    
    # feed predictor with test data and print results
    res = sess.run(pred, feed_dict={x: test_x})
    print("Weight(s): ")
    print(sess.run(W))
    print("Predidctions for: ")
    print(test_x)
    print("is: ")
    print(res)
    print('100Km/h = 62.13712mph')

# tensorboard.exe --logdir=".\tensorboard_example" --port 9000

epoch:  0  cost:  4084.49
Weight(s): 
[[ 0.6212129]]
Predidctions for: 
[[0.0], [100]]
is: 
[[  0.       ]
 [ 62.1212883]]
100Km/h = 62.13712mph
